In [23]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [24]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except RuntimeError as e:
        print(e)

In [25]:
class LinearDeepQNetwork(keras.Model):
    def __init__(self,
                 lr,
                 n_actions,
                 input_dims):
        super(LinearDeepQNetwork, self).__init__()
        
        self.fc1 = layers.Dense(128, activation='relu', input_shape=input_dims)
        self.fc2 = layers.Dense( n_actions, activation=None)
        self.flatten = layers.Flatten() 
        self.optimizer = keras.optimizers.Adam(learning_rate=lr)
        self.loss = keras.losses.MeanSquaredError()
    
    def call(self, inputs, training=False, **kwargs):
        with tf.device('/GPU:0'):  

            x = self.fc1(inputs)
            x = self.fc2(x)
            return x

In [26]:
class Agent():
    def __init__(self,
                 input_dims,
                 n_actions,
                 gamma=0.99,
                 lr=0.0001,
                 initial_epsilon=1,
                 epsilon_decay=1e-4,
                 final_epsilon=0.01):
        self.lr = lr
        self.n_actions = n_actions
        self.input_dims = input_dims
        self.gamma = gamma
        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon
        self.action_space = [i for i in range(self.n_actions)]
        
        self.Q = LinearDeepQNetwork(self.lr, self.n_actions, self.input_dims)
        
    def choose_action(self, obs):
            if np.random.random() > self.epsilon:
                state = tf.convert_to_tensor([obs], dtype=tf.float32)
                actions = self.Q(state)
                actions = tf.squeeze(actions)
                action = tf.argmax(actions)
                # print('All Actions: ', actions)
                # print('Selected Action: ', action.numpy())
                action = int(action.numpy())
            else:
                action = np.random.choice(self.action_space)
            
            return action
    
    def decrement_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)
        
    def learn(self, state, action, reward, next_state):
        with tf.GradientTape() as tape:
            states = tf.convert_to_tensor([state], dtype=tf.float32)
            actions = tf.convert_to_tensor([action], dtype=tf.int32)
            rewards = tf.convert_to_tensor([reward], dtype=tf.float32)
            next_states = tf.convert_to_tensor([next_state], dtype=tf.float32)
            
            # print(f'states {states.shape}, next_states {next_states.shape}')
            q_pred = self.Q.call(states)
#             print(f'q_pred: {q_pred.shape}')
            q_next = tf.reduce_max(self.Q.call(next_states))
            q_next_max = tf.reduce_max(q_next)
            # print(f'q_next_max: {q_next_max.shape}')        
            q_target = rewards + self.gamma * q_next_max
            # print(f'q_target: {q_target.shape}')
            loss = self.Q.loss(q_pred, q_target)
            
        gradients = tape.gradient(loss, self.Q.trainable_weights)
        self.Q.optimizer.apply_gradients(zip(gradients, self.Q.trainable_weights))
        self.decrement_epsilon()

In [27]:
if __name__ == '__main__':
    env = gym.make('CartPole-v1')
    nb_episodes = 10000
    scores = []
    eps_history = []
    agent = Agent(input_dims= env.observation_space.shape,
                  n_actions=env.action_space.n,
                  lr=0.0001
                  )
    avg_score =0
    for i in range (nb_episodes):
        if avg_score > 100 : 
            break
        else:
            score = 0
            done = False
            obs, _ = env.reset()
            while not done:
                action = agent.choose_action(obs)
                next_obs, reward, terminated,truncated, _ = env.step(action)
                score+= reward
                agent.learn(obs, action, reward, next_obs)
                obs = next_obs
                done = truncated or terminated
            scores.append(score)
            eps_history.append(agent.epsilon)
            
            if i % 100 == 0:
                avg_score = np.mean(scores[-100:])
                
                print('Episode', i, 'score %.1f avg score %.1f epsilon %.3f'% (score, avg_score,agent.epsilon) )
        
            

Episode 0 score 16.0 avg score 16.0 epsilon 0.998
Episode 100 score 13.0 avg score 22.2 epsilon 0.776
Episode 200 score 16.0 avg score 22.7 epsilon 0.550
Episode 300 score 11.0 avg score 22.8 epsilon 0.322
Episode 400 score 13.0 avg score 23.2 epsilon 0.089
Episode 500 score 13.0 avg score 23.9 epsilon 0.010
Episode 600 score 14.0 avg score 26.5 epsilon 0.010
Episode 700 score 52.0 avg score 26.6 epsilon 0.010
Episode 800 score 51.0 avg score 25.9 epsilon 0.010
Episode 900 score 25.0 avg score 26.7 epsilon 0.010
Episode 1000 score 27.0 avg score 25.6 epsilon 0.010
Episode 1100 score 10.0 avg score 27.4 epsilon 0.010
Episode 1200 score 9.0 avg score 27.2 epsilon 0.010
Episode 1300 score 36.0 avg score 27.1 epsilon 0.010
Episode 1400 score 18.0 avg score 27.9 epsilon 0.010
Episode 1500 score 10.0 avg score 26.2 epsilon 0.010
Episode 1600 score 23.0 avg score 27.9 epsilon 0.010
Episode 1700 score 31.0 avg score 30.3 epsilon 0.010
Episode 1800 score 13.0 avg score 24.9 epsilon 0.010
Episod

KeyboardInterrupt: 

In [ ]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg' if you prefer Qt
env = gym.make('CartPole-v1', render_mode = 'rgb_array')
obs, info = env.reset()

plt.ion()
fig, ax = plt.subplots(figsize=(8,8))
action_text = ax.text(510, 20, '', color='white', fontsize=12, bbox=dict(facecolor='blue', alpha=0.8))
actions = ['Left','Right']
img = ax.imshow(env.render())
rewards = 0
num_epochs= 10
for step in range(num_epochs):
    obs, info = env.reset()
    done = False
    while not done:
        action = agent.choose_action(obs)
        next_obs, reward, terminated, truncated, info = env.step(action)
        
        rewards += reward
        frame = env.render()
        img.set_data(frame)
        action_text.set_text(f'Step: {actions[action] }')

        fig.canvas.draw()
        fig.canvas.flush_events()
        done = terminated or truncated
        obs = next_obs

plt.ioff()  # Turn off interactive mode
# plt.show()  # Keep the window open after the animation finishes
plt.close()
env.close()

In [ ]:
plt.plot(scores)